In [ ]:
#librerias para el tratamiento de la API, transformacion en data frame, y tratamiento de fechas
import requests
import pandas as pd
from datetime import datetime, timedelta

#librerias para conectar la tabla con redshift
import psycopg2

#libreria para tratar las credenciales
import json


In [ ]:
# Leer el archivo de credenciales
with open('credentials.json') as file:
    credentials = json.load(file)

In [ ]:
#Parametros de la busqueda

query = 'ucrania OR ukraine OR Украина' #palabras clave para efectuar la busqueda en API
today = datetime.today()
yesterday = today - timedelta(days=1) # Obtener la fecha del dia de ayer
from_date = yesterday.strftime('%Y-%m-%d')
results_limit = 100 
api_key = credentials['api_key']
languages = ['ar', 'de','es', 'en', 'fr','ru', 'it']

# Funcion para realizar la conexion a la API
def get_articles(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print('Error al conectarse a la API:', e)
        return []
    else:
        data = response.json()
        articles = data['articles']
        return articles

# Crear una lista de diccionarios con los datos de cada artículo.
articles_list = []
for lang in languages:
    url = f"https://newsapi.org/v2/everything?q={query}&language={lang}&from={from_date}&pageSize={results_limit}&apiKey={api_key}"
    articles = get_articles(url)
    for article in articles:
        article_data = {
            'author': article['author'],
            'title': article['title'],
            'description': article['description'],
            'url': article['url'],
            'publishedAt': article['publishedAt'],
            'source': article['source']['name'],
            'language': lang
                        }
        articles_list.append(article_data)


In [ ]:
#convertir la lista de diccionarios en un DataFrame
df = pd.DataFrame(articles_list)

In [ ]:
#Ver el DF
df

In [ ]:
# Realizar conexion a Redshift mediante psycopg2
try:
    conn = psycopg2.connect(
    host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
    dbname=credentials['db_username'],
    user=credentials['db_password'],
    password='qjA21aB81Y',
    port='5439'
)
    print("Conexion con Redshift exitosa")
    
except Exception as e:
    print("Error al conectar con Redshift")
    print(e)


In [ ]:
# Crear un cursor para ejecutar sentencias SQL
cur = conn.cursor()

In [ ]:
# Eliminar la tabla si ya existe. Ejecutar solo en caso de error
#cur.execute('DROP TABLE IF EXISTS news_articles')

# Crear una tabla en Redshift, en caso de que no exista
table_name = 'news_articles'
cur.execute(f'''
CREATE TABLE IF NOT EXISTS {table_name} (
    author NVARCHAR(1000),
    title NVARCHAR(1000),
    description NVARCHAR(1000),
    url NVARCHAR(1000),
    published_at TIMESTAMP,
    source NVARCHAR(1000),
    language VARCHAR(2)
);
''')



In [ ]:
# Insertar los datos obtenidos de la API en la tabla
for article in articles_list:
    cur.execute(f'''
        INSERT INTO {table_name} (author,title, description, url, published_at, source, language)
        VALUES (
            %s, %s, %s, %s, %s, %s, %s
        );
    ''', (
        article['author'],
        article['title'],
        article['description'],
        article['url'],
        article['publishedAt'],
        article['source'],
        article['language']
    ))
# Hacer commit de la transacción y cerrar la conexión
conn.commit()
conn.close()



In [ ]:
#En caso de que el codigo para insertar los datos en la tabla de redshift fallen, ejecuto:
#cur.execute('ROLLBACK')
